In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Ensure TensorFlow is using GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Function to preprocess pixel data
def preprocess_pixel_data(pixel_string):
    pixel_values = np.array(list(map(float, pixel_string.split())), dtype=np.float32)
    return pixel_values.reshape(48, 48, 1)

# Load and preprocess training data
data = pd.read_csv('train_dataset.csv')
X = np.array([preprocess_pixel_data(row) for row in data['pixels']])

# Preprocess labels
y = to_categorical(data['emotion'], num_classes=7)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a simple CNN model for grayscale image classification
model_grayscale = Sequential()

# Add convolutional layers
model_grayscale.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model_grayscale.add(MaxPooling2D(pool_size=(2, 2)))

model_grayscale.add(Conv2D(64, (3, 3), activation='relu'))
model_grayscale.add(MaxPooling2D(pool_size=(2, 2)))

model_grayscale.add(Conv2D(128, (3, 3), activation='relu'))
model_grayscale.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output from the convolutional layers
model_grayscale.add(Flatten())

# Add fully connected layers
model_grayscale.add(Dense(128, activation='relu'))

# Output layer for 7 emotion classes
model_grayscale.add(Dense(7, activation='softmax'))

# Compile the model
model_grayscale.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_grayscale.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), batch_size=32)

# Load and preprocess test data
test_data = pd.read_csv('test_dataset.csv')
X_test_data = np.array([preprocess_pixel_data(row) for row in test_data['pixels']])

# Predict emotions for the test set
y_test_pred_prob_grayscale = model_grayscale.predict(X_test_data, batch_size=32)
y_test_pred_grayscale = np.argmax(y_test_pred_prob_grayscale, axis=1)

# Create a DataFrame for the submission
submission_df_grayscale = pd.DataFrame({
    'id': test_data['id'],
    'emotion': y_test_pred_grayscale
})

# Save the result to an Excel file
submission_df_grayscale.to_excel('submission_grayscale.xlsx', index=False)

print("Submission file created: 'submission_grayscale.xlsx'")


Num GPUs Available:  0
Epoch 1/200
125/125 [==============================] - 7s 54ms/step - loss: 2.7127 - accuracy: 0.2340 - val_loss: 1.7615 - val_accuracy: 0.2880
Epoch 2/200
125/125 [==============================] - 7s 53ms/step - loss: 1.7170 - accuracy: 0.3183 - val_loss: 1.6669 - val_accuracy: 0.3490
Epoch 3/200
125/125 [==============================] - 7s 55ms/step - loss: 1.5986 - accuracy: 0.3882 - val_loss: 1.6449 - val_accuracy: 0.3810
Epoch 4/200
125/125 [==============================] - 6s 51ms/step - loss: 1.4776 - accuracy: 0.4283 - val_loss: 1.5854 - val_accuracy: 0.3890
Epoch 5/200
125/125 [==============================] - 6s 51ms/step - loss: 1.3402 - accuracy: 0.4927 - val_loss: 1.5839 - val_accuracy: 0.4070
Epoch 6/200
125/125 [==============================] - 7s 53ms/step - loss: 1.2193 - accuracy: 0.5490 - val_loss: 1.6370 - val_accuracy: 0.4000
Epoch 7/200
125/125 [==============================] - 7s 54ms/step - loss: 1.0543 - accuracy: 0.6133 - val_loss: